# A1

In [1]:
from pyspark.sql import SparkSession 
spark_session = SparkSession.builder \
   .master("spark://192.168.2.251:7077") \
   .appName("alexanderSundquist_A1") \
   .config("spark.dynamicAllocation.enabled", True) \
   .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
   .config("spark.shuffle.service.enabled", False) \
   .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
   .config("spark.cores.max", 2) \
   .config("spark.driver.port",9999)\
   .config("spark.blockManager.port",10005)\
   .getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/28 09:43:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/28 09:43:29 WARN Utils: Service 'sparkDriver' could not bind on port 9999. Attempting port 10000.
25/02/28 09:43:29 WARN StandaloneSchedulerBackend: Dynamic allocation enabled without spark.executor.cores explicitly set, you may get more executors allocated than expected. It's recommended to set spark.executor.cores explicitly. Please check SPARK-30299 for more details.


In [2]:
# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

In [3]:
# Read Swedish and its English counterpart from HDFS
en_file = spark_context.textFile("hdfs://192.168.2.251:9000/data/europarl/europarl-v7.sv-en.en")
sv_file = spark_context.textFile("hdfs://192.168.2.251:9000/data/europarl/europarl-v7.sv-en.sv")

In [4]:
en_file.first()

'Resumption of the session'

In [5]:
sv_file.first()

'Återupptagande av sessionen'

In [6]:
print("Line count Swedish file", sv_file.count())
print("Line count English file", en_file.count())

Line count Swedish file 1862234


[Stage 3:=============================>                             (1 + 1) / 2]

Line count English file 1862234


In [7]:
# Count number of partitions 
en_file.getNumPartitions()

2

In [8]:
sv_file.getNumPartitions()

3

# A2

In [9]:
first_line = sv_file.first()
first_line_lower = first_line.lower()
print(first_line_lower)

[Stage 4:>                                                          (0 + 1) / 1]

återupptagande av sessionen


In [44]:
import re
def remove_special_char(file):
    return file.map(lambda line: re.sub("[^A-Za-z0-9 -åäöÅÄÖ]+",'', line))

In [45]:
def lower_case(file):
    return file.map(lambda line: line.lower())

In [46]:
def tokenize(file):
    return file.map(lambda line: line.split(" "))

In [47]:
sv_preprocessed = remove_special_char(sv_file)
sv_preprocessed = lower_case(sv_preprocessed)
sv_preprocessed = tokenize(sv_preprocessed)

en_preprocessed = remove_special_char(en_file)
en_preprocessed = lower_case(en_preprocessed)
en_preprocessed = tokenize(en_preprocessed)

print("Swedish preprocessed: ", sv_preprocessed.take(10))
print("English preprocessed: ", en_preprocessed.take(10))

Swedish preprocessed:  [['återupptagande', 'av', 'sessionen'], ['jag', 'förklarar', 'europaparlamentets', 'session', 'återupptagen', 'efter', 'avbrottet', 'den', '17', 'december.', 'jag', 'vill', 'på', 'nytt', 'önska', 'er', 'ett', 'gott', 'nytt', 'år', 'och', 'jag', 'hoppas', 'att', 'ni', 'haft', 'en', 'trevlig', 'semester.'], ['som', 'ni', 'kunnat', 'konstatera', 'ägde', '"den', 'stora', 'år', '2000-buggen"', 'aldrig', 'rum.', 'däremot', 'har', 'invånarna', 'i', 'ett', 'antal', 'av', 'våra', 'medlemsländer', 'drabbats', 'av', 'naturkatastrofer', 'som', 'verkligen', 'varit', 'förskräckliga.'], ['ni', 'har', 'begärt', 'en', 'debatt', 'i', 'ämnet', 'under', 'sammanträdesperiodens', 'kommande', 'dagar.'], ['till', 'dess', 'vill', 'jag', 'att', 'vi,', 'som', 'ett', 'antal', 'kolleger', 'begärt,', 'håller', 'en', 'tyst', 'minut', 'för', 'offren', 'för', 'bl.a.', 'stormarna', 'i', 'de', 'länder', 'i', 'europeiska', 'unionen', 'som', 'drabbats.'], ['jag', 'ber', 'er', 'resa', 'er', 'för', 'e

In [48]:
print("Line count Swedish file", sv_preprocessed.count())
print("Line count English file", en_preprocessed.count())

Line count Swedish file 1862234


[Stage 63:=============================>                            (1 + 1) / 2]

Line count English file 1862234


# A3

In [15]:
import re
def remove_special_char(file):
    return file.map(lambda line: re.sub("[^A-Za-z0-9 åäöÅÄÖ]+",'', line))

def lower_case(file):
    return file.map(lambda line: line.lower())

# Easier to use flatmap to break out of lists that are kept with map()
def tokenize(file):
    return file.flatMap(lambda line: line.split(" "))

sv_pre = remove_special_char(sv_file)
sv_pre = lower_case(sv_pre)
sv_pre = tokenize(sv_pre)

en_pre = remove_special_char(en_file)
en_pre = lower_case(en_pre)
en_pre = tokenize(en_pre)

In [16]:
# MapReduce to count occurences of each word

en_words = en_pre.map(lambda w: (w,1))
en_counts = en_words.reduceByKey(lambda a, b: a + b)

sv_words = sv_pre.map(lambda w: (w,1))
sv_counts = sv_words.reduceByKey(lambda a, b: a + b)

print(en_counts.take(5))
print(sv_counts.take(5))

[('resumption', 527), ('of', 1662006), ('i', 504521), ('declare', 1386), ('european', 270336)]


[Stage 11:======================================>                   (2 + 1) / 3]

[('session', 771), ('efter', 42231), ('avbrottet', 293), ('december', 6019), ('vill', 131230)]


In [17]:
# Calculate 10 most reacurring words in sv file

sorted_sv_counts = sv_counts.sortBy(lambda w: w[1], ascending=False)
sorted_sv_counts.take(10)

[('att', 1709939),
 ('och', 1350379),
 ('i', 1054253),
 ('det', 952991),
 ('som', 917591),
 ('för', 915081),
 ('av', 740724),
 ('är', 701842),
 ('en', 636829),
 ('vi', 546072)]

In [18]:
# Calculate total words in sv file

from operator import add

sv_words = sv_file.map(lambda line: line.split(" "))

sv_word_counts = sv_words.map(lambda w: len(w))

sv_total_words = sv_word_counts.reduce(add)

print(f"total words= {sv_total_words}")  

[Stage 20:======================================>                   (2 + 1) / 3]

total words= 41604741


In [19]:
# Calculate 10 most reacurring words in en file
sorted_en_counts = en_counts.sortBy(lambda w: w[1], ascending=False)
sorted_en_counts.take(10)

[('the', 3505175),
 ('of', 1662006),
 ('to', 1543746),
 ('and', 1318369),
 ('in', 1088902),
 ('that', 839083),
 ('is', 774941),
 ('a', 774545),
 ('for', 538192),
 ('we', 526488)]

In [20]:
# Calculate total words for en file
from operator import add

en_words = en_file.map(lambda line: line.split(" "))

en_word_counts = en_words.map(lambda w: len(w))

en_total_words = en_word_counts.reduce(add)

print(f"total words= {en_total_words}")  

[Stage 28:>                                                         (0 + 2) / 2]

total words= 45778381


# A4

In [21]:
sv_1 = sv_preprocessed.zipWithIndex()
en_1 = en_preprocessed.zipWithIndex()

In [22]:
sv_2 = sv_1.map(lambda x: (x[1], x[0]))
en_2 = en_1.map(lambda x: (x[1], x[0]))

In [23]:
sv_en_3 = sv_2.join(en_2)
sv_en_3.take(3)

[(148705,
  (['vi',
    'har',
    'tassat',
    'likt',
    'katten',
    'kring',
    'het',
    'gröt',
    'i',
    'den',
    'frågan',
    'alltför',
    'länge'],
   ['we',
    'have',
    'pussyfooted',
    'on',
    'that',
    'issue',
    'for',
    'far',
    'too',
    'long'])),
 (351195,
  (['jag', 'tackar', 'ledamoten', 'för', 'hans', 'frågor'],
   ['i', 'thank', 'the', 'honourable', 'member', 'for', 'his', 'questions'])),
 (444455,
  (['',
    '',
    'herr',
    'talman',
    'imorgon',
    'kommer',
    'vi',
    'att',
    'rösta',
    'om',
    'ett',
    'betänkande',
    'som',
    'inte',
    'längre',
    'handlar',
    'om',
    'själva',
    'ursprungsproblemet',
    'nämligen',
    'att',
    'minska',
    'luftföroreningarna',
    'genom',
    'en',
    'kvalitativ',
    'och',
    'kvantitativ',
    'begränsning',
    'av',
    'transittrafiken',
    'genom',
    'österrike'],
   ['',
    'mr',
    'president',
    'tomorrow',
    'we',
    'shall',
    'b

In [71]:
sv_en_4 = sv_en_3.filter(lambda x: x[1][1] != [''] and x[1][0] != [''])

In [72]:
# Calculate total words in sv file

from operator import add

word_count = sv_en_4.map(lambda w: len(w))

total_words = word_count.reduce(add)

print(f"total words= {total_words}")  

[Stage 93:==============================================>           (4 + 1) / 5]

total words= 3693754


In [73]:
print("Line count sv en removed empty lines", sv_en_4.count())


[Stage 95:==============================================>           (4 + 1) / 5]

Line count sv en removed empty lines 1846877


In [59]:
print("Line count sv en not removed empty lines: ", sv_en_3.count())

[Stage 75:==============================================>           (4 + 1) / 5]

Line count sv en not removed empty lines:  1862234


In [75]:
print(sv_en_3.count()-sv_en_4.count())

[Stage 100:=============================================>           (4 + 1) / 5]

15357


In [55]:
sv_file.filter(lambda x: x == "").count()

2462

In [69]:
sv_preprocessed.filter(lambda x: x == ['']).count()

2462

In [74]:
en_file.filter(lambda x: x == "").count()

11349

In [76]:
en_preprocessed.filter(lambda x: x == ['']).count()

11349

In [42]:
sv_file.take(3)

['Återupptagande av sessionen',
 'Jag förklarar Europaparlamentets session återupptagen efter avbrottet den 17 december. Jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester.',
 'Som ni kunnat konstatera ägde "den stora år 2000-buggen" aldrig rum. Däremot har invånarna i ett antal av våra medlemsländer drabbats av naturkatastrofer som verkligen varit förskräckliga.']

In [43]:
sv_preprocessed.take(3)

[['återupptagande', 'av', 'sessionen'],
 ['jag',
  'förklarar',
  'europaparlamentets',
  'session',
  'återupptagen',
  'efter',
  'avbrottet',
  'den',
  '17',
  'december',
  'jag',
  'vill',
  'på',
  'nytt',
  'önska',
  'er',
  'ett',
  'gott',
  'nytt',
  'år',
  'och',
  'jag',
  'hoppas',
  'att',
  'ni',
  'haft',
  'en',
  'trevlig',
  'semester'],
 ['som',
  'ni',
  'kunnat',
  'konstatera',
  'ägde',
  'den',
  'stora',
  'år',
  '2000buggen',
  'aldrig',
  'rum',
  'däremot',
  'har',
  'invånarna',
  'i',
  'ett',
  'antal',
  'av',
  'våra',
  'medlemsländer',
  'drabbats',
  'av',
  'naturkatastrofer',
  'som',
  'verkligen',
  'varit',
  'förskräckliga']]

# A5

In [62]:
sv_en_5 = sv_en_4.filter(lambda x: len(x[1][1]) < 10 and len(x[1][0]) < 10) 
sv_en_5.count()

210755

# A6

In [63]:
sv_en_6 = sv_en_5.filter(lambda x: len(x[1][1]) == len(x[1][0])) 
print(f"Line count sv_en_6 {sv_en_6.count()}")

[Stage 81:==============================================>           (4 + 1) / 5]

Line count sv_en_6 76296


# A7

In [66]:
sv_en_6.take(3)

[(51365, ([''], [''])),
 (53675,
  (['jag', 'förklarar', 'omröstningen', 'avslutad'],
   ['that', 'concludes', 'the', 'vote'])),
 (67310,
  (['detta', 'var', 'mitt', 'svar'], ['this', 'concludes', 'my', 'response']))]

In [65]:
sv_en_7 = sv_en_6.filter(lambda x: zip(x[1][0], x[1][1]))
sv_en_7.take(3)

[(28000,
  (['de', 'behöver', 'denna', 'plan', 'för', 'att', 'fortsätta'],
   ['they', 'need', 'this', 'scheme', 'to', 'go', 'ahead'])),
 (69075,
  (['jag', 'förklarar', 'debatten', 'avslutad'],
   ['the', 'debate', 'is', 'closed'])),
 (70435,
  (['samstämmighet',
    'och',
    'öppenhet',
    'är',
    'också',
    'viktiga',
    'principer',
    'för',
    'emas'],
   ['unanimity',
    'and',
    'openness',
    'are',
    'also',
    'important',
    'principles',
    'for',
    'emas']))]

In [ ]:
def zip_lists(list_1, list_2):
    translation_list = []
    for last, first in zip(list_1, list_2):
    pair_list = [last,first]
    translation_list.append(pair_list)
    return translation_list